## Periodic pattern mining on canadian TV logs
<img src="skmine_series.png" alt="logo" style="width: 60%;"/>

### The problem, informally
Let's take a simple example. 

Imagine you set an alarm to wake up every day around 7:30AM, and go to work. Sometimes you wake up a bit earlier (your body anticipates on the alarm), and sometimes a bit later, for example if you press the "snooze" button and refuse to face the fact that you have to wake up.

In python we can load those "wake up" events as logs, and store them in a [pandas.Series](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html), like

In [1]:
import datetime as dt
import pandas as pd
one_day = 60 * 24  # a day in minutes
minutes = [0, one_day - 1, one_day * 2 - 1, one_day * 3, one_day * 4 + 2, one_day * 7]

S = pd.Series("wake up", index=minutes)
start = dt.datetime.strptime("16/04/2020 07:30", "%d/%m/%Y %H:%M")
S.index = S.index.map(lambda e: start + dt.timedelta(minutes=e))
S.index = S.index.round("min")  # minutes as the lowest unit of difference
S

2020-04-16 07:30:00    wake up
2020-04-17 07:29:00    wake up
2020-04-18 07:29:00    wake up
2020-04-19 07:30:00    wake up
2020-04-20 07:32:00    wake up
2020-04-23 07:30:00    wake up
dtype: object

In [2]:
S.index.to_numpy()

array(['2020-04-16T07:30:00.000000000', '2020-04-17T07:29:00.000000000',
       '2020-04-18T07:29:00.000000000', '2020-04-19T07:30:00.000000000',
       '2020-04-20T07:32:00.000000000', '2020-04-23T07:30:00.000000000'],
      dtype='datetime64[ns]')

We can see the wake-up time is not exactly the same every day, but overall a consistent regular pattern seems to emerge.

Now imagine that in addition to wake up times, we also have records of other daily activities (meals, work, household chores, etc.), and that rather than a handful of days, those records span several years and make up several thousands of events

**How would you be able to detect regularities in the data ?**

### Introduction to periodic pattern mining
Periodic pattern mining aims at exploiting regularities not only about `what happens` by finding coordinated event occurrences, but also about `when it happens` and `how it happens`, by **finding consistent inter-occurrence timeintervals**.

Next, we introduce the concept of cycles

#### The cycle : a building block for periodic pattern mining
Here is an explicit example of a cycle

<img src="cycle_color.png" alt="cycle" style="width: 60%;"/>

This definition, while being relatively simple, is general enough to allow us to find regularities in different types of logs

#### Handling noise in our timestamps

Needless to say, it would be too easy if events in our data were equally spaced. As data often comes noisy, we have to be fault tolerant, and allow small errors to sneak into our cycles. 

That's the role of `shift corrections`, which capture the small deviations from perfectly regular periodic repetitions, and allow to reconstruct the (noisy) original sequence of events, using the following relation
<img src="shifts.png" alt="shifts" style="width: 60%;"/>

#### A tiny example with scikit-mine
`scikit-mine` offers a `PeriodicCycleMiner`, out of the box.
You can use it to **detect regularities, in the form of cycles**, in the input data. 

These regularities are submitted to an MDL criterion, so that we do not mistakenly include redundant occurences, nor forget to consider other intervals that would sumarize our data in a better way.

MDL offers a framework to find `the best set of cycles`, i.e the set that gives the most succint representation of the data. And `as humans, we often like to deal with non-redundant, well organized data`.

In [3]:
from skmine.periodic import PeriodicCycleMiner
import pandas as pd
S_doc = pd.Series("ring_a_bell", [10, 20, 32, 40, 60, 79, 100, 240])
pcm = PeriodicCycleMiner().fit(S_doc)
pcm.discover()

 ---- COLLECTION STATS (Total=1 nb_simple=1)
Code length patterns (1): 30.826182
Code length residuals (3): 23.555247
-- Total code length = 54.381429 (86.575343% of 62.813992)



/home/tbetton/Documents/F-WIN/scikit-mine/skmine/periodic/cycles.py:20: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


,t0,event,length_major,period_major,cost,type
0,20,(ring_a_bell)[r=5 p=20],5,20,30.826182,simple


In [4]:
from skmine.periodic import PeriodicCycleMiner
import pandas as pd

In [5]:
pcm = PeriodicCycleMiner().fit(S)

 ---- COLLECTION STATS (Total=1 nb_simple=1)
Code length patterns (1): 67.885186
Code length residuals (1): 15.884194
-- Total code length = 83.769381 (87.895949% of 95.305166)



In [6]:
pcm.discover()

,t0,event,length_major,period_major,cost,type
0,2020-04-16 07:30:00,(wake up)[r=5 p=8643],5,1 days 00:00:30,67.885186,simple


You can see one cycle has been extracted for our event `wake up`. The cycle covers the entire business week, but not the last monday separated by the weekend

It has a length of 5 and a period close to 1 day, as expected.

Also, note that we "lost" some information here. Our period of 1 day offers the best summary for this data.
Accessing the little "shifts" as encountered in original data is also possible, with an extra argument in our `.discover` call

The last column named `dE` contains a list of shifts to apply to our cycle in case we want to reconstruct the original data. Trailing zeros have been removed for efficiency, and their values are `relative to the period`, but we can see there is:
 * a -90 second shift between the 1st and 2nd entry (1day30s - 90s later = waking up at 7:29 on tuesday)
 * a 30 second shift between the 2nd and 3rd entry (1day30s - 30s later = still waking up at 7:29 on wednesday)
 * an 30 second shift between the 3rd and 4th entry (back to 7:30 on thursday)
 * an 90 second shift between the 4th and 5th entry (1day 30s + 90s later = waking up at 7:32 on friday)

Also note that we can get the "uncovered" events, called `redisuals`

In [7]:
pcm.get_residuals()

,time,event
0,2020-04-23 07:30:00,wake up


This way `pcm` does not store all the data, but has all information needed to reconstruct it entirely !!

In [8]:
pcm.reconstruct(0)

,time,event
0,2020-04-16 07:30:00,wake up
1,2020-04-17 07:30:30,wake up
2,2020-04-18 07:31:00,wake up
3,2020-04-19 07:31:30,wake up
4,2020-04-20 07:32:00,wake up


### An example with Canadian TV programs
#### Fetching logs from canadian TV

In this section we are going to load some event logs of TV programs (the `WHAT`), indexed by their broadcast timestamps (the `WHEN`).

`PeriodicCycleMiner` is here to help us discovering regularities (the `HOW`)

In [9]:
from skmine.datasets import fetch_canadian_tv
from skmine.periodic import PeriodicCycleMiner

#### Searching for cycles in TV programs

Remember about the definition of cycles ?
Let's apply it to our TV programs

In our case

* $\alpha$ is the name of a TV program

* $r$ is the number of broadcasts (repetitions) for this TV program (inside this cycle)

* $p$ is the optimal time delta between broadcasts in this cycle. If a program is meant to be live everyday at 14:00PM, then $p$ is likely to be `1 day`

* $\tau$ is the first broadcast time in this cycle

* $dE$ are the shift corrections between the $p$ and the actual broadcast time of an event. If a TV program was scheduled at 8:30:00AM and it went on air at 8:30:23AM the same day, then we keep track of a `23 seconds shift`. This way we can summarize our data (via cycles), and reconstruct it (via shift corrections). 


Finally we are going to dig a little deeper into these cycles, to answer quite complex questions about our logs. We will see that cycles contains usefull information about our input data

In [10]:
ctv_logs = fetch_canadian_tv()
ctv_logs.head()

/home/tbetton/Documents/F-WIN/scikit-mine/skmine/datasets/periodic.py:117: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  s = pd.read_csv(p, **kwargs)


timestamp
2020-08-01 06:00:00            The Moblees
2020-08-01 06:11:00    Big Block Sing Song
2020-08-01 06:13:00    Big Block Sing Song
2020-08-01 06:15:00               CBC Kids
2020-08-01 06:15:00               CBC Kids
Name: canadian_tv, dtype: string

Compute only simple cycles on ctv_logs with the karg *complex=False* : 

In [11]:
pcm = PeriodicCycleMiner().fit(ctv_logs, complex=False)
pcm.discover()

 ---- COLLECTION STATS (Total=231 nb_simple=231)
Code length patterns (231): 13607.850841
Code length residuals (507): 11994.387863
-- Total code length = 25602.238704 (71.289653% of 35912.979983)



,t0,event,length_major,period_major,cost,type
0,2020-08-01 05:00:00,(Grand Designs)[r=31 p=8640],31,1 days 00:00:00,101.455576,simple
1,2020-08-01 07:43:00,(CBC Kids)[r=26 p=8640],26,1 days 00:00:00,154.406032,simple
2,2020-08-28 00:00:00,(Schitt's Creek)[r=8 p=180],8,0 days 00:30:00,62.620856,simple
3,2020-08-01 06:59:00,(CBC Kids)[r=11 p=25920],11,3 days 00:00:00,93.039502,simple
4,2020-08-05 00:30:00,(Kim's Convenience)[r=7 p=180],7,0 days 00:30:00,60.844246,simple
...,...,...,...,...,...,...
226,2020-08-11 14:00:00,(Jamie & Jimmy's Food Fight Club)[r=4 p=8640],4,1 days 00:00:00,55.702844,simple
227,2020-08-01 11:25:00,(CBC Kids)[r=4 p=86004],4,9 days 22:54:00,56.979765,simple
228,2020-08-17 10:01:00,(CBC Kids)[r=5 p=8640],5,1 days 00:00:00,57.235173,simple
229,2020-08-03 07:23:00,(CBC Kids)[r=5 p=8640],5,1 days 00:00:00,57.235173,simple


Compute simple and complex (with horizontal and vertical combinations) cycles on ctv_logs : 

In [12]:
pcm = PeriodicCycleMiner().fit(ctv_logs)

 ---- COLLECTION STATS (Total=115 nb_simple=61 nb_concat=47 nb_other=6 nb_nested=1)
Code length patterns (115): 10880.109756
Code length residuals (454): 10805.204927
-- Total code length = 21685.314682 (60.382944% of 35912.979983)



In [13]:
cycles = pcm.discover()
cycles

,t0,event,length_major,period_major,cost,type
0,2020-08-01 05:00:00,(Grand Designs)[r=31 p=8640],31,1 days 00:00:00,101.455576,simple
1,2020-08-03 04:00:00,((CBC News: The National [d=2880] CBC News Net...,4,7 days 00:00:00,472.886395,other
2,2020-08-04 00:30:00,((This Hour Has 22 Minutes)[r=7 p=180] [d=5382...,3,7 days 00:00:00,255.643584,other
3,2020-08-04 07:00:00,((CBC Kids [d=258] CBC Kids)[r=4 p=60480])[r=4...,4,1 days 00:00:00,178.142750,other
4,2020-08-01 08:09:00,(CBC Kids [d=300] CBC Kids [d=6] Kingdom Force...,5,7 days 00:00:00,188.381406,concat
...,...,...,...,...,...,...
110,2020-08-04 08:58:00,(CBC Kids)[r=4 p=34272],4,3 days 23:12:00,55.149034,simple
111,2020-08-17 07:23:00,(Big Block Sing Song)[r=5 p=8643],5,1 days 00:00:30,75.235039,simple
112,2020-08-05 08:59:00,(CBC Kids [d=558] Daniel Tiger's Neighbourhood...,3,7 days 00:00:00,94.546656,concat
113,2020-08-10 10:59:00,(CBC Kids [d=2724] News)[r=4 p=60480],4,7 days 00:00:00,94.856042,concat


`Note` : no need to worry for the warning, it's here to notify duplicate event/timestamp pairs have been found

Now that we have our cycles in a [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html), we can play with the pandas API and answer questions about our logs

#### Did I find cycles for the TV show "Arthurt Shorts"

In [14]:
cycles

,t0,event,length_major,period_major,cost,type
0,2020-08-01 05:00:00,(Grand Designs)[r=31 p=8640],31,1 days 00:00:00,101.455576,simple
1,2020-08-03 04:00:00,((CBC News: The National [d=2880] CBC News Net...,4,7 days 00:00:00,472.886395,other
2,2020-08-04 00:30:00,((This Hour Has 22 Minutes)[r=7 p=180] [d=5382...,3,7 days 00:00:00,255.643584,other
3,2020-08-04 07:00:00,((CBC Kids [d=258] CBC Kids)[r=4 p=60480])[r=4...,4,1 days 00:00:00,178.142750,other
4,2020-08-01 08:09:00,(CBC Kids [d=300] CBC Kids [d=6] Kingdom Force...,5,7 days 00:00:00,188.381406,concat
...,...,...,...,...,...,...
110,2020-08-04 08:58:00,(CBC Kids)[r=4 p=34272],4,3 days 23:12:00,55.149034,simple
111,2020-08-17 07:23:00,(Big Block Sing Song)[r=5 p=8643],5,1 days 00:00:30,75.235039,simple
112,2020-08-05 08:59:00,(CBC Kids [d=558] Daniel Tiger's Neighbourhood...,3,7 days 00:00:00,94.546656,concat
113,2020-08-10 10:59:00,(CBC Kids [d=2724] News)[r=4 p=60480],4,7 days 00:00:00,94.856042,concat


In [15]:
cycles[cycles["event"].apply(lambda x: "Arthur Shorts" in x)]

,t0,event,length_major,period_major,cost,type
7,2020-08-03 09:48:00,((Arthur Shorts [d=78] CBC Kids)[r=5 p=8640])[...,3,6 days 23:59:00,193.816983,other
36,2020-08-01 11:00:00,(Arthur Shorts)[r=5 p=60480],5,7 days 00:00:00,54.030035,simple
81,2020-08-24 09:48:00,(Arthur Shorts [d=258] CBC Kids)[r=4 p=8640],4,1 days 00:00:00,114.108878,concat


#### What are the top 10 longest cycles ?

In [16]:
cycles.nlargest(10, ["length_major"])

,t0,event,length_major,period_major,cost,type
0,2020-08-01 05:00:00,(Grand Designs)[r=31 p=8640],31,1 days 00:00:00,101.455576,simple
68,2020-08-17 07:22:00,(CBC Kids)[r=12 p=8640],12,1 days 00:00:00,111.340664,simple
10,2020-08-28 00:00:00,(Schitt's Creek)[r=8 p=180],8,0 days 00:30:00,62.620856,simple
16,2020-08-05 00:30:00,(Kim's Convenience)[r=7 p=180],7,0 days 00:30:00,60.844246,simple
17,2020-08-26 00:30:00,(Kim's Convenience)[r=7 p=180],7,0 days 00:30:00,60.844246,simple
18,2020-08-06 00:30:00,(Mr. D)[r=7 p=180],7,0 days 00:30:00,60.844246,simple
19,2020-08-07 00:30:00,(Schitt's Creek)[r=7 p=180],7,0 days 00:30:00,60.844246,simple
29,2020-08-23 07:27:00,(CBC Kids)[r=6 p=8640],6,1 days 00:00:00,58.858624,simple
50,2020-08-11 08:59:00,(CBC Kids)[r=6 p=17280],6,2 days 00:00:00,58.549426,simple
4,2020-08-01 08:09:00,(CBC Kids [d=300] CBC Kids [d=6] Kingdom Force...,5,7 days 00:00:00,188.381406,concat


In [17]:
pcm.reconstruct(1)

,time,event
0,2020-08-03 04:00:00,CBC News: The National
1,2020-08-03 12:00:00,CBC News Network
2,2020-08-04 04:00:00,CBC News: The National
3,2020-08-04 12:00:00,CBC News Network
4,2020-08-05 04:00:00,CBC News: The National
...,...,...
115,2020-08-26 11:00:00,Dragons' Den
116,2020-08-27 10:59:00,CBC Kids
117,2020-08-27 11:00:00,Dragons' Den
118,2020-08-28 10:59:00,CBC Kids


In [18]:
pcm.get_residuals()

,time,event
0,2020-08-04 09:35:00,Arthur Shorts
1,2020-08-01 13:00:00,Bondi Vet
2,2020-08-25 17:00:00,Heartland
3,2020-08-20 03:00:00,Mr. D
4,2020-08-31 15:00:00,Jamie's Super Foods
...,...,...
449,2020-08-11 08:33:00,CBC Kids
450,2020-08-02 07:43:00,Rusty Rivets
451,2020-08-27 08:07:00,CBC Kids
452,2020-08-07 09:00:00,Ollie: The Boy Who Became What He Ate


#### what are the 10 most unpunctual TV programs ?
For this we are going to :
 1. extract the shift corrections along with other informations about our cycles
 2. compute the sum of the absolute values for the shift corrections, for every cycles
 3. get the 10 biggest sums

In [19]:
# full_cycles = pcm.discover()
# full_cycles.head()

In [20]:
# def absolute_sum(*args):
#     return sum(map(abs, *args))

# # level 0 is the name of the TV program
# shift_sums = full_cycles["dE"].map(absolute_sum).groupby(level=[0]).sum()
# shift_sums.nlargest(10)

<!-- #### what are the 10 most unpunctual TV programs ?
For this we are going to :
 1. extract the shift corrections along with other informations about our cycles
 2. compute the sum of the absolute values for the shift corrections, for every cycles
 3. get the 10 biggest sums -->

In [21]:
# def absolute_sum(*args):
#     return sum(map(abs, *args))

# # level 0 is the name of the TV program
# shift_sums = full_cycles["dE"].map(absolute_sum).groupby(level=[0]).sum()
# shift_sums.nlargest(10)

#### What TV programs have been broadcasted every day for at least 5 days straight?
Let's make use of the [pandas.DataFrame.query](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html) method to express our question in an SQL-like syntax

In [22]:
# cycles.query('length_major >= 5 and period_major >= 3600', engine='python')
# # cycles.query('length >= 5 and period == "1 days"', engine='python')

#### What TV programs are broadcast only on business days ?
From the previous query we see we have a lot of 5-length cycles, with periods of 1 day.
An intuition is that these cycles take place on business days. Let's confirm this by considering cycles with
 1. start timestamps on mondays
 2. periods of roughly 1 day  

In [23]:
# monday_starts = cycles[cycles.start.dt.weekday == 0]  # start on monday
# monday_starts.query('length == 5 and period == "1 days"', engine='python')

References
----------

1.
    Galbrun, E & Cellier, P & Tatti, N & Termier, A & Crémilleux, B
    "Mining Periodic Pattern with a MDL Criterion"

2.
    Galbrun, E
    "The Minimum Description Length Principle for Pattern Mining : A survey"

3. 
    Termier, A
    ["Periodic pattern mining"](http://people.irisa.fr/Alexandre.Termier/dmv/DMV_Periodic_patterns.pdf) 